In [ ]:
# LAN-Score: Learning to Adapt Noise for Score-based Point Cloud Denoising

这个笔记本演示了如何使用LAN（Learning to Adapt Noise）方法来增强Score-based点云去噪模型的效果。


In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from utils.misc import *
from utils.transforms import *
from utils.denoise import *
from models.lan_score import *


In [ ]:
## 1. 加载预训练模型


In [ ]:
# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# 加载预训练模型
ckpt_path = './pretrained/ckpt.pt'
ckpt = torch.load(ckpt_path, map_location=device)
model = LANScoreNet(ckpt['args']).to(device)
model.load_state_dict(ckpt['state_dict'], strict=False)
model.eval()
print("模型加载完成")


In [ ]:
## 2. 加载点云数据


In [ ]:
# 加载点云数据
example_dir = './data/examples/PUNet_10000_poisson_0.02'
example_files = [f for f in os.listdir(example_dir) if f.endswith('.xyz')]
print(f"找到 {len(example_files)} 个点云文件")

# 选择一个点云文件
example_file = example_files[0]
pcl_noisy = torch.FloatTensor(np.loadtxt(os.path.join(example_dir, example_file)))
pcl_noisy, center, scale = NormalizeUnitSphere.normalize(pcl_noisy)
pcl_noisy = pcl_noisy.to(device)
print(f"点云形状: {pcl_noisy.shape}")


In [ ]:
## 3. 使用传统Score-based方法去噪


In [ ]:
# 使用传统Score-based方法去噪
with torch.no_grad():
    pcl_denoised_orig = patch_based_denoise(
        model=model,
        pcl_noisy=pcl_noisy,
        ld_step_size=0.2,
        ld_num_steps=30,
        step_decay=0.95,
        seed_k=3,
        denoise_knn=4
    )
print("传统Score-based去噪完成")


In [ ]:
## 4. 使用LAN方法去噪


In [ ]:
# 创建LAN模块
lan = LAN(pcl_noisy.unsqueeze(0).shape).to(device)
optimizer = torch.optim.Adam(lan.parameters(), lr=1e-4)

# 选择自监督损失函数
loss_func = zsn2n_loss_func  # 或者使用 nbr2nbr_loss_func

# 优化LAN参数
inner_loop = 20
loss_values = []
print("开始优化LAN参数...")
for i in tqdm(range(inner_loop)):
    optimizer.zero_grad()
    loss = loss_func(pcl_noisy.unsqueeze(0), model, lan)
    loss.backward()
    optimizer.step()
    loss_values.append(loss.item())
print("LAN参数优化完成")

# 绘制损失曲线
plt.figure(figsize=(10, 5))
plt.plot(loss_values)
plt.title('LAN优化损失曲线')
plt.xlabel('迭代次数')
plt.ylabel('损失')
plt.grid(True)
plt.show()


In [ ]:
# 使用LAN方法去噪
with torch.no_grad():
    # 查看LAN适应后的点云
    pcl_adapted = lan(pcl_noisy.unsqueeze(0))[0]
    
    # 使用LAN进行去噪
    pcl_denoised_lan = patch_based_denoise(
        model=model,
        pcl_noisy=pcl_noisy,
        ld_step_size=0.2,
        ld_num_steps=30,
        step_decay=0.95,
        seed_k=3,
        denoise_knn=4,
        lan=lan
    )
print("LAN方法去噪完成")


In [ ]:
## 5. 比较结果


In [ ]:
# 计算Chamfer距离
from models.utils import chamfer_distance_unit_sphere

# 加载干净的点云（如果有）
clean_dir = './data/PUNet/test/10000_poisson'
clean_file = example_file.replace('_noisy', '')
if os.path.exists(os.path.join(clean_dir, clean_file)):
    pcl_clean = torch.FloatTensor(np.loadtxt(os.path.join(clean_dir, clean_file)))
    pcl_clean, _, _ = NormalizeUnitSphere.normalize(pcl_clean)
    pcl_clean = pcl_clean.to(device)
    
    # 计算Chamfer距离
    cd_orig = chamfer_distance_unit_sphere(pcl_denoised_orig.unsqueeze(0), pcl_clean.unsqueeze(0))[0].item()
    cd_lan = chamfer_distance_unit_sphere(pcl_denoised_lan.unsqueeze(0), pcl_clean.unsqueeze(0))[0].item()
    
    print(f"原始Score-based方法的Chamfer距离: {cd_orig:.6f}")
    print(f"LAN方法的Chamfer距离: {cd_lan:.6f}")
    print(f"改进: {cd_orig - cd_lan:.6f} ({(cd_orig - cd_lan) / cd_orig * 100:.2f}%)")


In [ ]:
## 6. 保存结果


In [ ]:
# 保存结果
output_dir = './results'
os.makedirs(output_dir, exist_ok=True)

# 反归一化
pcl_noisy_denorm = pcl_noisy.cpu() * scale + center
pcl_adapted_denorm = pcl_adapted.cpu() * scale + center
pcl_denoised_orig_denorm = pcl_denoised_orig.cpu() * scale + center
pcl_denoised_lan_denorm = pcl_denoised_lan.cpu() * scale + center

# 保存点云
np.savetxt(os.path.join(output_dir, 'noisy.xyz'), pcl_noisy_denorm.numpy(), fmt='%.8f')
np.savetxt(os.path.join(output_dir, 'adapted.xyz'), pcl_adapted_denorm.numpy(), fmt='%.8f')
np.savetxt(os.path.join(output_dir, 'denoised_orig.xyz'), pcl_denoised_orig_denorm.numpy(), fmt='%.8f')
np.savetxt(os.path.join(output_dir, 'denoised_lan.xyz'), pcl_denoised_lan_denorm.numpy(), fmt='%.8f')

print(f"结果已保存到 {output_dir} 目录")


In [ ]:
## 7. 可视化比较（需要额外的可视化库）


In [ ]:
# 尝试使用Open3D进行可视化
try:
    import open3d as o3d
    
    def visualize_point_cloud(points, color=[0.5, 0.5, 0.5]):
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
        pcd.paint_uniform_color(color)
        return pcd
    
    # 创建点云对象
    pcd_noisy = visualize_point_cloud(pcl_noisy_denorm.numpy(), [1, 0, 0])  # 红色
    pcd_adapted = visualize_point_cloud(pcl_adapted_denorm.numpy(), [0, 1, 0])  # 绿色
    pcd_denoised_orig = visualize_point_cloud(pcl_denoised_orig_denorm.numpy(), [0, 0, 1])  # 蓝色
    pcd_denoised_lan = visualize_point_cloud(pcl_denoised_lan_denorm.numpy(), [1, 1, 0])  # 黄色
    
    # 可视化
    print("红色: 噪声点云")
    o3d.visualization.draw_geometries([pcd_noisy])
    
    print("绿色: LAN适应后的点云")
    o3d.visualization.draw_geometries([pcd_adapted])
    
    print("蓝色: 原始Score-based方法去噪结果")
    o3d.visualization.draw_geometries([pcd_denoised_orig])
    
    print("黄色: LAN方法去噪结果")
    o3d.visualization.draw_geometries([pcd_denoised_lan])
    
    print("比较原始方法(蓝色)和LAN方法(黄色)")
    o3d.visualization.draw_geometries([pcd_denoised_orig, pcd_denoised_lan])
    
except ImportError:
    print("未安装Open3D库，无法进行可视化。可以使用其他工具查看保存的点云文件。")
